## 予備実験3: 試作1
- ~sampleデータを増やすような処理手順でマルチラベルを実行してみる.~
    - ~未知語対応や, 先にword2vecでのエンべディングを実施し, 未知語を省く対応を行うなど.~
1. predリンクの特徴量生成 + クラスタリング
    - 試作1: word2vecをpredリンクにシンプルに実施
    - 試作2: keyデータ + objデータのベクトルから演算することでpredリンクの特徴量生成を実施
2. クラスタリングで先に特徴量的に近い関係リンクのまとまりを作り, sampleデータを増やす処理手順を行う
    - 先にまとまりを作ることで, ある程度似通ったデータ同士のマルチラベル予測を実施する.

In [84]:
import glob
import pandas as pd
import os
import numpy as np

from gensim.models import Word2Vec
from sklearn.preprocessing import LabelBinarizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier

In [50]:
# data/result/に格納されている全csvファイルの読み込み
csv_files = glob.glob(os.path.join("../../data/result", "*.csv"))
df_list = []
for file in csv_files:
    tmp_df = pd.read_csv(file)
    df_list.append(tmp_df)
df = pd.concat(df_list, ignore_index=True)
df.shape

(56796, 3)

In [51]:
# predの要素で, propertyが含まれる値を返す
df_prop = df.query('pred.str.contains("property")', engine='python').dropna()

# 'wikiPage'を含まないpropertyを返す
df_prop = df_prop[~df_prop['pred'].str.contains('wikiPage')]

# '画像'を含まないpropertyを返す
df_prop = df_prop[~df_prop['pred'].str.contains('画像')]

df_prop.shape

(2500, 3)

In [52]:
# モデル読み込み
model = Word2Vec.load('../../Models/japanese-word2vec-model-builder/word2vec.gensim.model')

In [54]:
# モデルを利用した文字のエンべディング
## 未知語に関しては, 今回はNaNで対応
def vectorize(model, word):
    try:
        output = model.wv[word]
        return output
    except:
        return np.nan

In [55]:
# obj要素に関してはLOD同士が繋がっているため, 基本的にURIで記述されている.
# また, (県の魚:〇〇)の要素や数値データが入っている場合もあるため, それぞれ最後尾の要素を値として扱う. 
def preprocessing(obj):
    if type(obj) != str:
        output = obj
    elif len(obj.split("：")) != 1:
        output = obj.split("：")[-1]
    else:
        output = obj.split("/")[-1]
    return output

### 関係リンク(pred_link)を基にしたクラスタ作成を実施する.

In [56]:
# embeding

cp_df_prop = df_prop.copy()
pred_list = cp_df_prop['pred'].map(lambda x:preprocessing(x))
cp_df_prop.loc[:,'pred_vec'] = pred_list.map(lambda x:vectorize(model, x))

In [57]:
# ベクトル化前のデータとベクトル化後のデータの結合 + 未知語のNaNの排除

df_vec = pd.merge(df_prop, cp_df_prop, left_index=True, right_on=df_prop.index)
df_vec = df_vec.drop(["key_0","key_y","pred_y","obj_y"], axis=1)
df_vec = df_vec.dropna().reset_index(drop=True)

df_vec.shape

(1521, 4)

### predリンクの特徴量からクラスタを作成していく

In [71]:
from sklearn.cluster import KMeans
from collections import defaultdict

In [81]:
n_clusters = 4
kmeans_model = KMeans(n_clusters=n_clusters, random_state=42)
kmeans_model.fit(clusters_input)

KMeans(n_clusters=4, random_state=42)

In [82]:
cluster_labels = kmeans_model.labels_
cluster_to_words = defaultdict(list)
for cluster_id, word in zip(cluster_labels, df_vec.pred_x):
    cluster_to_words[cluster_id].append(word)

In [83]:
for words in cluster_to_words.values():
    print(set(words))

{'http://ja.dbpedia.org/property/before', 'http://ja.dbpedia.org/property/north', 'http://ja.dbpedia.org/property/south', 'http://ja.dbpedia.org/property/name', 'http://ja.dbpedia.org/property/years', 'http://ja.dbpedia.org/property/east', 'http://ja.dbpedia.org/property/after', 'http://ja.dbpedia.org/property/label', 'http://ja.dbpedia.org/property/date', 'http://ja.dbpedia.org/property/west'}
{'http://ja.dbpedia.org/property/float', 'http://ja.dbpedia.org/property/y', 'http://ja.dbpedia.org/property/image', 'http://ja.dbpedia.org/property/d', 'http://ja.dbpedia.org/property/x', 'http://ja.dbpedia.org/property/color', 'http://ja.dbpedia.org/property/section', 'http://ja.dbpedia.org/property/s', 'http://ja.dbpedia.org/property/q', 'http://ja.dbpedia.org/property/height', 'http://ja.dbpedia.org/property/direction', 'http://ja.dbpedia.org/property/species', 'http://ja.dbpedia.org/property/コード', 'http://ja.dbpedia.org/property/centre', 'http://ja.dbpedia.org/property/code', 'http://ja.dbp